In [1]:
!pip install yfinance plotly pandas


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
import io
from datetime import datetime


In [3]:
print("Upload your portfolio CSV (symbol,quantity,avg_price)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
df_port = pd.read_csv(io.BytesIO(uploaded[file_name]))

df_port


Upload your portfolio CSV (symbol,quantity,avg_price)


Saving portfolio.csv to portfolio.csv


,symbol,quantity,avg_price
0,TCS.NS,10,3200
1,RELIANCE.NS,5,2750
2,INFY.NS,8,1500
3,HDFCBANK.NS,6,1600


In [4]:
symbols = df_port['symbol'].tolist()

# Download latest data
data = yf.download(tickers=symbols, period="5d", interval="1d", progress=False)

# Handle multi-index safely
if isinstance(data.columns, pd.MultiIndex):
    last_close = data['Close'].iloc[-1]
else:
    last_close = data['Close']

# Map market price
df_port['market_price'] = df_port['symbol'].map(last_close.to_dict())

# P&L Calculations
df_port['notional'] = df_port['market_price'] * df_port['quantity']
df_port['cost'] = df_port['avg_price'] * df_port['quantity']
df_port['pnl'] = df_port['notional'] - df_port['cost']
df_port['pnl_pct'] = (df_port['pnl'] / df_port['cost']) * 100

# Sensitivity (±1% impact)
df_port['impact_1pct'] = df_port['notional'] * 0.01

df_port


/tmp/ipython-input-2053125319.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=symbols, period="5d", interval="1d", progress=False)


,symbol,quantity,avg_price,market_price,notional,cost,pnl,pnl_pct,impact_1pct
0,TCS.NS,10,3200,3137.500000,31375.000000,32000,-625.000000,-1.953125,313.750000
1,RELIANCE.NS,5,2750,1567.500000,7837.500000,13750,-5912.500000,-43.000000,78.375000
2,INFY.NS,8,1500,1560.099976,12480.799805,12000,480.799805,4.006665,124.807998
3,HDFCBANK.NS,6,1600,1007.599976,6045.599854,9600,-3554.400146,-37.025002,60.455999


In [5]:
print("=== Portfolio Summary ===")
print(df_port)

print("\nTotal Cost:", df_port['cost'].sum())
print("Total Market Value:", df_port['notional'].sum())
print("Total P&L:", df_port['pnl'].sum())
print("Portfolio Risk (1% move):", df_port['impact_1pct'].sum())


=== Portfolio Summary ===
        symbol  quantity  avg_price  market_price      notional   cost  \
0       TCS.NS        10       3200   3137.500000  31375.000000  32000   
1  RELIANCE.NS         5       2750   1567.500000   7837.500000  13750   
2      INFY.NS         8       1500   1560.099976  12480.799805  12000   
3  HDFCBANK.NS         6       1600   1007.599976   6045.599854   9600   

           pnl    pnl_pct  impact_1pct  
0  -625.000000  -1.953125   313.750000  
1 -5912.500000 -43.000000    78.375000  
2   480.799805   4.006665   124.807998  
3 -3554.400146 -37.025002    60.455999  

Total Cost: 67350
Total Market Value: 57738.899658203125
Total P&L: -9611.100341796875
Portfolio Risk (1% move): 577.3889965820313


In [6]:
fig = px.bar(
    df_port,
    x="symbol",
    y="pnl",
    title="P&L by Stock",
    text="pnl"
)
fig.show()


In [7]:
fig2 = px.bar(
    df_port,
    x="symbol",
    y="impact_1pct",
    title="1% Price Move Impact per Position"
)
fig2.show()


In [8]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f"portfolio_results_{timestamp}.csv"

df_port.to_csv(output_file, index=False)
files.download(output_file)

print("Saved:", output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved: portfolio_results_20251130_074803.csv
